<h2><font size="3" color="#008B8B"><strong>Principais objetivos: </strong></font></h2>
<ul>
    <li><font size="2" color="#1f77b4">Coletar dados API do SIGE.</font></li>
    <li><font size="2" color="#1f77b4">Corverter e formatar datas (timezone, formato pandas).</font></li>
    <li><font size="2" color="#1f77b4">Adicionar colunas (predio e grupo).</font></li>
    <li><font size="2" color="#1f77b4">Salvar em arquivo csv os dados brutos.</font></li>

</ul>

In [1]:
import pandas as pd
from dateutil.parser import isoparse
import pytz
import datetime
import httpx


In [2]:
transductor_mapping = {
   1 : {"ip": "164.41.20.230", "serial_number": "2115505", "group": 2, "name": "UAC 2"},
   2 : {"ip": "164.41.20.231", "serial_number": "2115506", "group": 2, "name": "UAC 3"},
   3 : {"ip": "164.41.20.233", "serial_number": "2246535", "group": 1, "name": "UED 2"},
   4 : {"ip": "164.41.20.234", "serial_number": "2246536", "group": 1, "name": "UED 3"},
   5 : {"ip": "164.41.20.236", "serial_number": "2249071", "group": 3, "name": "LDTEA 1"},
   6 : {"ip": "164.41.20.237", "serial_number": "2246534", "group": 3, "name": "LDTEA 2"},
   7 : {"ip": "164.41.20.238", "serial_number": "2246528", "group": 3, "name": "LDTEA 3"},
   8 : {"ip": "164.41.20.239", "serial_number": "2249072", "group": 3, "name": "LDTEA 4"},
   9 : {"ip": "164.41.20.241", "serial_number": "2246531", "group": 5, "name": "MASP 1"},
}


In [3]:
# Funcoes para coleta de dados da API do SIGE

def request_measurements(url, pk=None, start_date=None, end_date=None):
    params = {
        "transductor": pk,
        "start_date": start_date,
        "end_date": end_date,
    }
    params = {k: v for k, v in params.items() if v is not None}

    data = []
    while url:
        page_data, next_url = send_request(url, params)
        data.extend(page_data)
        url = next_url

    print("Finished data collection.")
    print(f"Total results received: {len(data)}")
    return data


def send_request(url, params=None):
    try:
        response = httpx.get(url, params=params)
        response.raise_for_status()

        json_response = response.json()
        results = json_response.get("results", [])
        next_url = json_response.get("next")

        return results, next_url
    except httpx.HTTPError as exc:
        print(f"HTTP Exception for {exc.request.url} - {exc}")
        raise

def get_group(id_transductor):
    return transductor_mapping.get(id_transductor, {}).get("group")


def get_building(id_transductor):
    return transductor_mapping.get(id_transductor, {}).get("name")


def convert_timezone(date):
    br_timezone = pytz.timezone("America/Sao_Paulo")

    if isinstance(date, str):
        date = isoparse(date)

    date_tz = date.astimezone(br_timezone)
    return date_tz.strftime("%Y-%m-%d %H:%M:%S")


In [4]:
# Carregar dados da API do SIGE e transformar em dataframe

url = "http://localhost:8000/energy_consumption/"
# url = "http://164.41.98.17:443/energy_consumption/"
data = request_measurements(url)

df_raw = pd.DataFrame(data)
df_raw.rename(columns={"collection_date": "date_time"}, inplace=True)
print(df_raw.shape)
df_raw.head()


Finished data collection.
Total results received: 116071
(116071, 7)


,id,transductor,active_consumption,active_generated,reactive_inductive,reactive_capacitive,date_time
0,116311,4,0.00,0.00,0.00,0.00,2023-10-11T18:15:42.531905-03:00
1,116310,5,0.00,-0.04,0.00,-0.57,2023-10-11T18:15:42.521777-03:00
2,116309,1,0.02,-0.02,0.31,-0.90,2023-10-11T18:15:42.510817-03:00
3,116308,6,0.00,-0.03,0.00,-0.57,2023-10-11T18:15:42.499538-03:00
4,116307,3,0.00,0.00,0.00,0.00,2023-10-11T18:15:42.489512-03:00


In [5]:
# Formatar datas e adicionar colunas

df = df_raw.copy()
df["date_time"] = df_raw["date_time"].apply(isoparse)
df["date_time"] = df["date_time"].apply(convert_timezone)
df["date_time"] = pd.to_datetime(df["date_time"])

df["group"] = df["transductor"].apply(get_group)
df["building"] = df["transductor"].apply(get_building)

df.drop(columns=["id"], inplace=True)
df.set_index("date_time", inplace=True)
df.sort_index(inplace=True)

print(df.index.min(), df.index.max())
print(df.shape)
df.head()


2023-05-29 21:15:03 2023-10-11 18:15:42
(116071, 7)


,transductor,active_consumption,active_generated,reactive_inductive,reactive_capacitive,group,building
date_time,,,,,,,
2023-05-29 21:15:03,1,0.0,0.0,0.0,0.0,2,UAC 2
2023-05-29 21:30:03,3,0.0,0.0,0.0,0.0,1,UED 2
2023-05-29 21:30:03,2,0.0,0.0,0.0,0.0,2,UAC 3
2023-05-29 21:30:03,1,0.0,0.0,0.0,0.0,2,UAC 2
2023-05-29 21:45:02,2,0.0,0.0,0.0,0.0,2,UAC 3


In [6]:
# Salvar arquivo csv
df.to_csv("../dataset/energy_production_data.csv")
